In [1]:
import pandas as pd

# Load your CSV file into a DataFrame
df_final = pd.read_csv('updated_csv_file.csv')

C:\Users\vinay\AppData\Local\Temp\ipykernel_11836\669674392.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv('updated_csv_file.csv')


In [2]:
num_rows = df_final.shape[0]  # Number of rows
num_columns = df_final.shape[1]  # Number of columns

# Get the last column name
last_column_name = df_final.columns[-1]
first_column_name=df_final.columns[0]

print(f'Number of rows: {num_rows}')
print(f'Number of columns: {num_columns}')
print(f'First column name: {first_column_name}')
print(f'Last column name: {last_column_name}')

Number of rows: 931
Number of columns: 16394
First column name: PATIENT_ID
Last column name: SUBTYPE


In [3]:



# DATA PRE PROCESSING


import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Step 2: Check for missing values and handle them

# Separate numeric and non-numeric columns
numeric_cols = df_final.select_dtypes(include=['float64', 'int64']).columns
non_numeric_cols = df_final.select_dtypes(exclude=['float64', 'int64']).columns

# Fill missing values for numeric columns with the column mean
df_final[numeric_cols] = df_final[numeric_cols].fillna(df_final[numeric_cols].mean())

# Fill missing values for non-numeric columns with the most frequent value (mode)
for col in non_numeric_cols:
    df_final[col].fillna(df_final[col].mode()[0], inplace=True)

# Step 3: Scale only the numeric columns
scaler = StandardScaler()
df_final_scaled = df_final.copy()  # Create a copy of the DataFrame to hold scaled values
df_final_scaled[numeric_cols] = scaler.fit_transform(df_final[numeric_cols])  # Only scale the numeric columns

# Step 4: Separate features (X) and target (y)
# X = all columns except the last one (Subtype), retaining both numeric and non-numeric columns
X = df_final_scaled.iloc[:, :-1]  # Features
y = df_final_scaled.iloc[:, -1]   # Target (Subtype)

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Output the shapes of the training and testing sets
print("Preproccessed Successfully")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

C:\Users\vinay\AppData\Local\Temp\ipykernel_11836\72013851.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final[col].fillna(df_final[col].mode()[0], inplace=True)


Preproccessed Successfully
X_train shape: (744, 16393)
X_test shape: (187, 16393)
y_train shape: (744,)
y_test shape: (187,)


In [8]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import ADASYN
import numpy as np

# Step 1: Ensure numeric columns are selected
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Extract numeric columns from X_train and X_test
X_train_numeric = X_train[numeric_cols]
X_test_numeric = X_test[numeric_cols]

# Step 2: Dimensionality Reduction with PCA (retain 95% variance)
pca = PCA(n_components=0.95)  # Automatically selects components retaining 95% variance
X_train_pca = pca.fit_transform(X_train_numeric)
X_test_pca = pca.transform(X_test_numeric)

print("Reduced X_train shape after PCA:", X_train_pca.shape)

# Step 3: Handle class imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_pca, y_train)

# Step 4: Expanded Grid Search for Hyperparameter Tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10],          # Wider range of C values for regularization
    'gamma': [0.0001, 0.00001, 0.000001],  # Explore smaller gamma values
    'kernel': ['rbf']                   # Use only the RBF kernel
}

# Step 5: Define class weights
class_weights = {'BRCA_Basal': 1, 'BRCA_Her2': 10, 'BRCA_LumA': 1, 'BRCA_LumB': 3}

# Perform GridSearchCV with cross-validation
grid = GridSearchCV(SVC(probability=True, class_weight=class_weights, random_state=42),
                    param_grid, refit=True, verbose=2, cv=5, return_train_score=True)
grid.fit(X_train_resampled, y_train_resampled)

# Step 6: Get Best Parameters and Evaluate Model
print("Best Parameters from Grid Search:", grid.best_params_)

# Cross-validation scores
cv_results = grid.cv_results_
train_scores = cv_results['mean_train_score']
test_scores = cv_results['mean_test_score']

print("\nCross-Validation Results:")
print("Average Training Score:", np.mean(train_scores))
print("Average Validation Score:", np.mean(test_scores))

# Check for Overfitting
if np.mean(train_scores) > np.mean(test_scores) + 0.05:  # Allowing a small margin
    print("\nWarning: The model might be overfitting!")
else:
    print("\nThe model does not appear to be overfitting.")

y_pred_grid = grid.predict(X_test_pca)

print("\nSVM Results after Hyperparameter Tuning:")
print("Accuracy:", accuracy_score(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid, zero_division=0))

# Step 7: Evaluate Confusion Matrix and Additional Metrics
cm = confusion_matrix(y_test, y_pred_grid)
print("\nConfusion Matrix for SVM:\n", cm)

mcc = matthews_corrcoef(y_test, y_pred_grid)
y_proba_svm = grid.predict_proba(X_test_pca)
roc_auc = roc_auc_score(y_test, y_proba_svm, multi_class='ovr')

print("\nSVM MCC after Grid Search:", mcc)
print("SVM AUC after Grid Search:", roc_auc)


Reduced X_train shape after PCA: (744, 541)
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, gamma=0.0001, kernel=rbf; total time=   3.0s
[CV] END ...................C=0.01, gamma=0.0001, kernel=rbf; total time=   3.0s
[CV] END ...................C=0.01, gamma=0.0001, kernel=rbf; total time=   3.2s
[CV] END ...................C=0.01, gamma=0.0001, kernel=rbf; total time=   3.0s
[CV] END ...................C=0.01, gamma=0.0001, kernel=rbf; total time=   3.4s
[CV] END ....................C=0.01, gamma=1e-05, kernel=rbf; total time=   2.9s
[CV] END ....................C=0.01, gamma=1e-05, kernel=rbf; total time=   3.0s
[CV] END ....................C=0.01, gamma=1e-05, kernel=rbf; total time=   3.0s
[CV] END ....................C=0.01, gamma=1e-05, kernel=rbf; total time=   3.0s
[CV] END ....................C=0.01, gamma=1e-05, kernel=rbf; total time=   3.0s
[CV] END ....................C=0.01, gamma=1e-06, kernel=rbf; total time=   2.8s
[CV]

In [12]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE
import numpy as np

# Step 1: Ensure numeric columns are selected
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Extract numeric columns from X_train and X_test
X_train_numeric = X_train[numeric_cols]
X_test_numeric = X_test[numeric_cols]

# Step 2: Dimensionality Reduction with PCA (retain 95% variance)
pca = PCA(n_components=0.95)  # Automatically selects components retaining 95% variance
X_train_pca = pca.fit_transform(X_train_numeric)
X_test_pca = pca.transform(X_test_numeric)

print("Reduced X_train shape after PCA:", X_train_pca.shape)

# Step 3: Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pca, y_train)

print(f"Shape after SMOTE Resampling: {X_train_resampled.shape}")

# Step 4: Expanded Grid Search for Hyperparameter Tuning
param_grid = {
    'C': [0.1, 1, 10, 100],          # Wider range of C values for regularization
    'gamma': [0.001, 0.0001, 0.00001],  # Explore smaller gamma values
    'degree': [2, 3],                 # Degrees for the polynomial kernel
    'kernel': ['poly']                # Use the polynomial kernel
}

# Step 5: Define class weights
class_weights = {'BRCA_Basal': 1, 'BRCA_Her2': 15, 'BRCA_LumA': 1, 'BRCA_LumB': 7}

# Perform GridSearchCV with return_train_score=True to capture training scores
grid = GridSearchCV(SVC(probability=True, class_weight=class_weights, random_state=42),
                    param_grid, refit=True, verbose=2, return_train_score=True)
grid.fit(X_train_resampled, y_train_resampled)

# Step 6: Get Best Parameters and Evaluate Model
print("Best Parameters from Grid Search:", grid.best_params_)

y_pred_grid = grid.predict(X_test_pca)

print("\nSVM Results after Hyperparameter Tuning:")
print("Accuracy:", accuracy_score(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid, zero_division=0))

# Step 7: Evaluate Confusion Matrix and Additional Metrics
cm = confusion_matrix(y_test, y_pred_grid)
print("\nConfusion Matrix for SVM:\n", cm)

mcc = matthews_corrcoef(y_test, y_pred_grid)
y_proba_svm = grid.predict_proba(X_test_pca)
roc_auc = roc_auc_score(y_test, y_proba_svm, multi_class='ovr')

print("\nSVM MCC after Grid Search:", mcc)
print("SVM AUC after Grid Search:", roc_auc)

# Step 8: Cross-Validation for Overfitting Check
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(grid.best_estimator_, X_train_resampled, y_train_resampled, cv=cv, scoring='accuracy')

print("\nCross-Validation Accuracy Scores:", cv_scores)
print("Mean Cross-Validation Accuracy:", np.mean(cv_scores))
print("Standard Deviation of Cross-Validation Accuracy:", np.std(cv_scores))

# Step 9: Check for Overfitting
train_accuracy = grid.best_score_
mean_cv_accuracy = np.mean(cv_scores)

print("\nTraining Accuracy from Grid Search:", train_accuracy)
print("Cross-Validation Accuracy:", mean_cv_accuracy)

if train_accuracy > mean_cv_accuracy + 0.05:  # Threshold for significant overfitting
    print("The model shows signs of overfitting.")
else:
    print("The model does not show significant overfitting.")


Reduced X_train shape after PCA: (744, 541)
Shape after SMOTE Resampling: (1544, 541)
Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END ..........C=0.1, degree=2, gamma=0.001, kernel=poly; total time=   1.9s
[CV] END ..........C=0.1, degree=2, gamma=0.001, kernel=poly; total time=   1.9s
[CV] END ..........C=0.1, degree=2, gamma=0.001, kernel=poly; total time=   2.0s
[CV] END ..........C=0.1, degree=2, gamma=0.001, kernel=poly; total time=   2.2s
[CV] END ..........C=0.1, degree=2, gamma=0.001, kernel=poly; total time=   2.1s
[CV] END .........C=0.1, degree=2, gamma=0.0001, kernel=poly; total time=   2.4s
[CV] END .........C=0.1, degree=2, gamma=0.0001, kernel=poly; total time=   2.4s
[CV] END .........C=0.1, degree=2, gamma=0.0001, kernel=poly; total time=   2.7s
[CV] END .........C=0.1, degree=2, gamma=0.0001, kernel=poly; total time=   2.4s
[CV] END .........C=0.1, degree=2, gamma=0.0001, kernel=poly; total time=   2.6s
[CV] END ..........C=0.1, degree=2, gamma=

In [13]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import ADASYN
import numpy as np

# Step 1: Ensure numeric columns are selected
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Extract numeric columns from X_train and X_test
X_train_numeric = X_train[numeric_cols]
X_test_numeric = X_test[numeric_cols]

# Step 2: Dimensionality Reduction with PCA (retain 95% variance)
pca = PCA(n_components=0.95)  # Automatically selects components retaining 95% variance
X_train_pca = pca.fit_transform(X_train_numeric)
X_test_pca = pca.transform(X_test_numeric)

print("Reduced X_train shape after PCA:", X_train_pca.shape)

# Step 3: Handle class imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_pca, y_train)

# Step 4: Expanded Grid Search for Hyperparameter Tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10],          # Wider range of C values for regularization
    'gamma': [0.0001, 0.00001, 0.000001],  # Explore smaller gamma values
    'kernel': ['sigmoid']                   # Use 'sigmoid' kernel
}

# Step 5: Define class weights
class_weights = {'BRCA_Basal': 1, 'BRCA_Her2': 10, 'BRCA_LumA': 1, 'BRCA_LumB': 3}

# Perform GridSearchCV with cross-validation
grid = GridSearchCV(SVC(probability=True, class_weight=class_weights, random_state=42),
                    param_grid, refit=True, verbose=2, cv=5, return_train_score=True)
grid.fit(X_train_resampled, y_train_resampled)

# Step 6: Get Best Parameters and Evaluate Model
print("Best Parameters from Grid Search:", grid.best_params_)

# Cross-validation scores
cv_results = grid.cv_results_
train_scores = cv_results['mean_train_score']
test_scores = cv_results['mean_test_score']

print("\nCross-Validation Results:")
print("Average Training Score:", np.mean(train_scores))
print("Average Validation Score:", np.mean(test_scores))

# Check for Overfitting
if np.mean(train_scores) > np.mean(test_scores) + 0.05:  # Allowing a small margin
    print("\nWarning: The model might be overfitting!")
else:
    print("\nThe model does not appear to be overfitting.")

y_pred_grid = grid.predict(X_test_pca)

print("\nSVM Results after Hyperparameter Tuning:")
print("Accuracy:", accuracy_score(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid, zero_division=0))

# Step 7: Evaluate Confusion Matrix and Additional Metrics
cm = confusion_matrix(y_test, y_pred_grid)
print("\nConfusion Matrix for SVM:\n", cm)

mcc = matthews_corrcoef(y_test, y_pred_grid)
y_proba_svm = grid.predict_proba(X_test_pca)
roc_auc = roc_auc_score(y_test, y_proba_svm, multi_class='ovr')

print("\nSVM MCC after Grid Search:", mcc)
print("SVM AUC after Grid Search:", roc_auc)


Reduced X_train shape after PCA: (744, 541)
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...............C=0.01, gamma=0.0001, kernel=sigmoid; total time=   3.0s
[CV] END ...............C=0.01, gamma=0.0001, kernel=sigmoid; total time=   2.6s
[CV] END ...............C=0.01, gamma=0.0001, kernel=sigmoid; total time=   2.7s
[CV] END ...............C=0.01, gamma=0.0001, kernel=sigmoid; total time=   9.2s
[CV] END ...............C=0.01, gamma=0.0001, kernel=sigmoid; total time=   3.4s
[CV] END ................C=0.01, gamma=1e-05, kernel=sigmoid; total time=   3.2s
[CV] END ................C=0.01, gamma=1e-05, kernel=sigmoid; total time=   2.9s
[CV] END ................C=0.01, gamma=1e-05, kernel=sigmoid; total time=   3.1s
[CV] END ................C=0.01, gamma=1e-05, kernel=sigmoid; total time=   3.0s
[CV] END ................C=0.01, gamma=1e-05, kernel=sigmoid; total time=   3.0s
[CV] END ................C=0.01, gamma=1e-06, kernel=sigmoid; total time=   3.0s
[CV]

In [21]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Custom utility kernel function
def utilitykernel(x1, x2, a, p=1):
    x1 = x1.flatten()
    x2 = x2.flatten()
    dot_product = np.dot(x1, x2)
    result = np.log(np.power(np.cosh(a * dot_product), p))
    max_value = 700
    return np.clip(result, -max_value, max_value)

# Custom Gram matrix computation
def gaussianKernelGramMatrix(X1, X2, a, K_function=utilitykernel):
    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            gram_matrix[i, j] = K_function(x1, x2, a)
    return gram_matrix

# Assuming X_train, X_test, y_train, and y_test are defined as DataFrames
# Preprocessing: Select only numeric columns
X_train_numeric = X_train.select_dtypes(include=[np.number])
X_test_numeric = X_test.select_dtypes(include=[np.number])

print("Preprocessing Complete. Shapes:")
print("X_train_numeric shape:", X_train_numeric.shape)
print("X_test_numeric shape:", X_test_numeric.shape)

# Step 1: Apply PCA
pca = PCA(n_components=0.95)  # Adjust number of components as needed
X_train_pca = pca.fit_transform(X_train_numeric)
X_test_pca = pca.transform(X_test_numeric)

print("Original X_train shape:", X_train_numeric.shape)
print("Reduced X_train shape after PCA:", X_train_pca.shape)

# Step 3: Generate Gram matrices using the custom kernel
a = 0.001
p = 2
X_train_gram = gaussianKernelGramMatrix(X_train_pca, X_train_pca, a)
X_test_gram = gaussianKernelGramMatrix(X_test_pca, X_train_pca, a)

# Step 4: Define parameter grid
param_grid = {
    'C': [0.01],
    'kernel': ['precomputed']
}

# Step 5: Initialize and fit GridSearchCV with SVM
grid = GridSearchCV(
    SVC(probability=True, class_weight='balanced', random_state=42),
    param_grid,
    refit=True,
    verbose=2,
    return_train_score=True,
    cv=5
)
grid.fit(X_train_gram, y_train)

# Step 6: Get best parameters
print("Best Parameters from Grid Search:", grid.best_params_)

# Step 7: Evaluate cross-validation results for overfitting
train_scores = grid.cv_results_['mean_train_score']
val_scores = grid.cv_results_['mean_test_score']
score_diff = train_scores - val_scores

print("\nCross-Validation Results:")
for i, C in enumerate(param_grid['C']):
    print(f"C={C}: Mean Training Score: {train_scores[i]:.4f}, Mean Validation Score: {val_scores[i]:.4f}, Difference: {score_diff[i]:.4f}")

# Check for overfitting
if any(diff > 0.1 for diff in score_diff):
    print("\nPotential overfitting detected. Significant gap between training and validation scores.")
else:
    print("\nNo significant overfitting detected. Training and validation scores are similar.")

# Step 8: Make predictions
y_pred_grid = grid.predict(X_test_gram)

# Step 9: Evaluate model performance
print("\nSVM Results after Hyperparameter Tuning:")
print("Accuracy:", accuracy_score(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid, zero_division=0))

# Step 10: Additional metrics
cm = confusion_matrix(y_test, y_pred_grid)
FP = cm.sum(axis=0) - np.diag(cm)
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

TPR = TP / (TP + FN)
TNR = TN / (TN + FP)

print("\nConfusion Matrix for SVM:\n", cm)
print("\nSensitivity (TPR) for SVM:", TPR)
print("Specificity (TNR) for SVM:", TNR)

mcc = matthews_corrcoef(y_test, y_pred_grid)
y_proba_svm = grid.predict_proba(X_test_gram)
roc_auc = roc_auc_score(y_test, y_proba_svm, multi_class='ovr')

print("\nSVM MCC after Grid Search:", mcc)
print("SVM AUC after Grid Search:", roc_auc)


Preprocessing Complete. Shapes:
X_train_numeric shape: (744, 16384)
X_test_numeric shape: (187, 16384)
Original X_train shape: (744, 16384)
Reduced X_train shape after PCA: (744, 541)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .........................C=0.01, kernel=precomputed; total time=   0.0s
[CV] END .........................C=0.01, kernel=precomputed; total time=   0.0s
[CV] END .........................C=0.01, kernel=precomputed; total time=   0.0s
[CV] END .........................C=0.01, kernel=precomputed; total time=   0.0s
[CV] END .........................C=0.01, kernel=precomputed; total time=   0.0s
Best Parameters from Grid Search: {'C': 0.01, 'kernel': 'precomputed'}

Cross-Validation Results:
C=0.01: Mean Training Score: 0.7389, Mean Validation Score: 0.6532, Difference: 0.0857

No significant overfitting detected. Training and validation scores are similar.

SVM Results after Hyperparameter Tuning:
Accuracy: 0.679144385026738
              p

In [33]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from imblearn.over_sampling import ADASYN
import numpy as np

# Step 1: Directly use the numeric data from X_train and X_test (assuming they're already numeric)
# If they are in pandas, you can convert to numpy using .values if needed
# In this case, we're assuming X_train and X_test are already numpy arrays.

# Step 2: Dimensionality Reduction with PCA (retain 95% variance)
pca = PCA(n_components=0.95)  # Automatically selects components retaining 95% variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print("Reduced X_train shape after PCA:", X_train_pca.shape)

# Step 3: Handle class imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_pca, y_train)

# Step 4: Define your custom utility kernel function
def utility_kernel(X, Y, p=1):
    # Custom utility kernel
    return 7*(np.dot(X, Y.T)) ** p   # Example: dot product kernel, you can modify it for other kernels

# Step 5: Expanded Grid Search for Hyperparameter Tuning
param_grid = {
    'C': [0.0001,0.001,0.01, 0.1, 1, 10],  # Wider range of C values for regularization
    'gamma': [0.0001, 0.00001, 0.000001],  # Explore smaller gamma values
    'kernel': [utility_kernel]  # Use the custom utility kernel
}

# Step 6: Define class weights
class_weights = {'BRCA_Basal': 1, 'BRCA_Her2': 10, 'BRCA_LumA': 1, 'BRCA_LumB': 3}

# Perform GridSearchCV with cross-validation
grid = GridSearchCV(SVC(probability=True, class_weight=class_weights, random_state=42),
                    param_grid, refit=True, verbose=2, cv=5, return_train_score=True)
grid.fit(X_train_resampled, y_train_resampled)

# Step 7: Get Best Parameters and Evaluate Model
print("Best Parameters from Grid Search:", grid.best_params_)

# Cross-validation scores
cv_results = grid.cv_results_
train_scores = cv_results['mean_train_score']
test_scores = cv_results['mean_test_score']

print("\nCross-Validation Results:")
print("Average Training Score:", np.mean(train_scores))
print("Average Validation Score:", np.mean(test_scores))

# Check for Overfitting
if np.mean(train_scores) > np.mean(test_scores) + 0.05:  # Allowing a small margin
    print("\nWarning: The model might be overfitting!")
else:
    print("\nThe model does not appear to be overfitting.")

y_pred_grid = grid.predict(X_test_pca)

print("\nSVM Results after Hyperparameter Tuning:")
print("Accuracy:", accuracy_score(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid, zero_division=0))

# Step 8: Evaluate Confusion Matrix and Additional Metrics
cm = confusion_matrix(y_test, y_pred_grid)
print("\nConfusion Matrix for SVM:\n", cm)

mcc = matthews_corrcoef(y_test, y_pred_grid)
y_proba_svm = grid.predict_proba(X_test_pca)
roc_auc = roc_auc_score(y_test, y_proba_svm, multi_class='ovr')

print("\nSVM MCC after Grid Search:", mcc)
print("SVM AUC after Grid Search:", roc_auc)


Reduced X_train shape after PCA: (744, 421)
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END C=0.0001, gamma=0.0001, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.1s
[CV] END C=0.0001, gamma=0.0001, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.0s
[CV] END C=0.0001, gamma=0.0001, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.1s
[CV] END C=0.0001, gamma=0.0001, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.0s
[CV] END C=0.0001, gamma=0.0001, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.1s
[CV] END C=0.0001, gamma=1e-05, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.1s
[CV] END C=0.0001, gamma=1e-05, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.0s
[CV] END C=0.0001, gamma=1e-05, kernel=<function utility_kernel at 0x000001D86AFEE340>; total time=   0.0s
[CV] END C=0.0001, gamma=1e-05, ke